In [1]:
%run ./passwords.ipynb

In [2]:
import pandas as pd
import sqlalchemy
import pyodbc




def Extract_from_mssql():
    
    try:
        total=0
        engine_mssql = sqlalchemy.create_engine(
                    f"mssql+pyodbc://{user}:{password}@{SERVERNAME}/{DB}?driver=ODBC+Driver+17+for+SQL+Server",
                    fast_executemany=True)
        txt='''SELECT SCHEMA_NAME(schema_id)+'.'+name AS SchemaTable FROM sys.tables
                where name not in('targets','BTCUSDT','index_logs') '''
        tables = pd.read_sql_query(txt,engine_mssql)
        for tab in tables["SchemaTable"]:
            txt=f"select*from {tab}"
            df = pd.read_sql_query(txt,engine_mssql)
            
            total=total+import_to_Postgresql(df,tab)
                
    except Exception as e:
        print("error")

    finally:
        print("")
        print("Result:")
        print(total,"rows have been totally imported from MS SQL server to PostgreSQL")
        
def import_to_Postgresql(df,tab):
    try:
        
        engine_Psql = sqlalchemy.create_engine(f'postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{server_pg}:{POSTGRES_PORT}/{POSTGRES_DBNAME}')
        print(f' {len(df)} rows have been imported to postgreSQL for table {tab}')
        
        df.to_sql(f'gala_{tab}',engine_Psql,if_exists='replace',index=False)
        print("success")
        return len(df)
        
    except Exception as e:
        print("error")    

In [3]:
Extract_from_mssql()

 7 rows have been imported to postgreSQL for table production.categories
success
 9 rows have been imported to postgreSQL for table production.brands
success
 321 rows have been imported to postgreSQL for table production.products
success
 1445 rows have been imported to postgreSQL for table sales.customers
success
 3 rows have been imported to postgreSQL for table sales.stores
success
 10 rows have been imported to postgreSQL for table sales.staffs
success
 1615 rows have been imported to postgreSQL for table sales.orders
success
 4722 rows have been imported to postgreSQL for table sales.order_items
success
 939 rows have been imported to postgreSQL for table production.stocks
success

Result:
9071 rows have been totally imported from MS SQL server to PostgreSQL
